In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import os


#---------------------------------
def check(img,a,b,k,sig):
    a = (a+1)//2 if a%2==1 else a//2
    b = (b+1)//2 if b%2==1 else b//2
    img = cv2.resize(img,(b,a))
    nxt = cv2.GaussianBlur(img,(k,k),sig)
    return nxt,a,b
#----------------------------------
'''
    Gaussian_Pyramid
    img = image, n = number of levels
    first level is considered as level 0
'''
def Gaussian_Pyramid(img,n,k=5,sig=1):
    pyramids = {}
    a,b = img.shape
    for level in range(n):
        pyramids[level] = cv2.GaussianBlur(img,(k,k),sig)
        a = a//2; b = b//2
        img = cv2.resize(pyramids[level],(b,a))
    return pyramids
'''
    Laplacian_Pyramid
    img = image, n = number of levels
    first level is considered as level 0
'''
def Laplacian_Pyramid(img,n,k=5,sig = 1):
    pyramids = {}
    a,b = img.shape    
    prev = cv2.GaussianBlur(img,(k,k),sig)
    for level in range(n):
        f1 = b; f2 = a
        nxt,a,b = check(prev,a,b,k,sig)
        lap = cv2.subtract(prev , cv2.resize(nxt,(f1,f2)))
        pyramids[level] = lap
        prev = nxt
    return pyramids
#---------------------------------------------
# the below function downsamples the 2 images img1 and img2
'''
img1 --- image1
img2 --- image2
n --- downsampling factor
k --- size of the gaussian filter to be applied to avoid aliasing 
sig --- standard deviation of the gaussian filter 
'''

def downsamp(img1,img2,n = 4,k=5,sig=1):
    a,b = img1.shape
    img1 = cv2.GaussianBlur(img1,(k,k),sig)
    img2 = cv2.GaussianBlur(img2,(k,k),sig)
    img1 = cv2.resize(img1,(b//n,a//n))
    img2 = cv2.resize(img2,(b//n,a//n))
    return img1,img2

#--------------------------------------------------------------------
#as we do not have the translation vector and angular velocity vector between image1 and image2, we perform a euclidean transform for an image
'''
image --- image to be transformed
t --- translation vector of shape (3,)
theta ---- angle to be rotated in radians  
'''
def euclidean_tranform(img,t,theta):
    matrix = np.array([[math.cos(theta), -1*math.sin(theta),t[0]],[math.sin(theta),math.cos(theta),t[1]],[0,0,1]]) #euclidean matrix
    m_max,n_max = img.shape
    img_mod = np.zeros(img.shape)
    for i in range(m_max):
        for j in range(n_max):
            point = np.matmul(matrix,np.array([[i],[j],[1]])) #tranforming each point
            point = point/point[2,:]                          #calculating homogenous point
            point = np.int32(np.round(point)) 
            if 0<=point[0,:]<m_max and 0<=point[1,:]<n_max:
                img_mod[point[0,:],point[1,:]] = img[i,j]  
    return img_mod                                          #return the modified image

#-----------------------------------------------------------------------
'''
absolute difference between 2 images
'''
def img_difference(img1,img2):
    diff = img1-img2
    diff = abs(diff)
    diff = np.int32(diff)
    return diff
#-----------------------------------------------------------------------
#code for image modification
'''
inputs  ref_img --- reference image I(t)
        img_unmod --- unmodified image I(t-1)
        U_img ---- (u,v) values matrix 
'''

def img_modification(ref_img,img_unmod,U_img):
    img_mod = np.zeros(img_unmod.shape)             #initializing the image modified
    x_max,y_max = img_unmod.shape
    for i in range(x_max):                          #for every (i,j) in the unmodified image
        for j in range(y_max):
            u1,v1 = np.int32(np.floor(U_img[i,j,:]))    #flooring (u,v)
            u2,v2 = np.int32(np.ceil(U_img[i,j,:]))     #ceiling (u,v)
            if (np.array([i-u1,i-u2])<x_max).all() and (np.array([i-u1,i-u2])>=0).all() and (np.array([j-v1,j-v2])<y_max).all() and (np.array([j-v1,j-v2])>=0).all():
                if  abs(ref_img[i,j]-img_unmod[i-u1,j-v1])<abs(ref_img[i,j]-img_unmod[i-u2,j-v2]): #comparing how close is the pixel intensity to that of the reference image at the same location
                    img_mod[i,j] = img_unmod[i-u1,j-v1]
                else:
                    img_mod[i,j] = img_unmod[i-u2,j-v2]
    return img_mod
#-------------------------------------------------------------------------
'''
code for the gradient of I
'''

def gradient_I(img):
    Ix = cv2.Sobel(img,-1,0,1) #sobel horizontal 
    Iy = cv2.Sobel(img,-1,1,0) # sobel vertical
    return Ix,Iy
#------------------------------------------------------------------------
#calculation of the required matrices and vectors for planar flow
'''
f --- focal length
(x,y) --- point coordinates
'''
def A(f,x,y):
    A_mat = np.array([[-1*f,0,x],[0,-1*f,y]])
    return A_mat

def B(f,x,y):
    B_mat = np.array([[(x*y)/f,-1*(f+x**2)/f,y],[(f+y**2)/f,-1*(x*y)/f,-1*x]])
    return B_mat

def r(f,x,y):
    r_vec = np.array([[x/f],[y/f],[1]])
    return r_vec

#--------------------------------------------------------------------------
#depending on the delta_k obtained per iteration delta_U_mat is calculated
'''
delta_k ---- delta k obtained per iteration 
(m_max,n_max) ---- shape of the delta_U_mat 
t --- translation vector
f ---- focal length
'''
def delta_U_mat(delta_k,m_max,n_max,t,f):
    del_u_mat = np.zeros((m_max,n_max,2))
    for i in range(m_max):
        for j in range(n_max):
            r_vec = r(f,i,j)
            one = np.matmul(A(f,i,j),t)
            two = np.matmul(one,r_vec.T)
            del_u = np.matmul(two,delta_k)
            del_u_mat[i,j,0] = del_u[0,:]
            del_u_mat[i,j,1] = del_u[1,:]
    return del_u_mat 

#------------------------------------------------------------------------
#planar flow code per level
'''
img1 --- I(t-1)
img2 --- I(t)
U_prev --- U matrix initialization
f --- focal length 
t --- translational vector
iter --- iterations 
'''
def planar_flow(img1,img2,U_prev,f,t,iter=10):
    Ix,Iy = gradient_I(img2)                                #gradient of the reference image
    m_max,n_max = img1.shape                                #shape of the image
    # U_prev = np.zeros((m_max,n_max,2))
    img1_mod = img_modification(img2,img1,U_prev)           #modified image based on U_prev
    for it in range(iter):
        #print(it)
        delta_I = img2-img1_mod                             #delta_I 
        L=0                                                 #LHS of the equation(present in the paper)
        R=0                                                 #RHS
        for i in range(1,m_max-1):
            for j in range(1,n_max-1):
                r_vec = r(f,i,j)                            #r vector calculation 
                grad_I = np.array([[Ix[i,j]],[Iy[i,j]]])    #gradient I vector
                At = np.matmul(A(f,i,j),t)                  #A*t calculation 
                L1 = np.matmul(r_vec,At.T)                  
                L2 = np.matmul(L1,grad_I)
                L3 = np.matmul(L2,L2.T)
                L = L + L3                                  #L calculation 
                R1 = L2*delta_I[i,j]                    
                R = R + R1                                  #R calculation 
        R = -1*R
        delta_k = np.matmul(np.linalg.pinv(L),R)            #delta_k calculation
        delta_U = delta_U_mat(delta_k,m_max,n_max,t,f)      #delta_U calculation
        U_prev = U_prev + delta_U                           #adding delta_U to U_prev
        img1_mod = img_modification(img2,img1,U_prev)       #modified image
    return U_prev,img1_mod                                  #returning U_prev and modified image
#-------------------------------------------------------------------------------

'''
'''

def hier_planar(img1,img2,f,t,no_of_levels=2,pyramid = 'Gaussian' ,iter_per_level = 4, k=3,sig = 1): 
    if pyramid == 'Gaussian':
        py_img1 = Gaussian_Pyramid(img1,no_of_levels,k,sig)                 #gaussian pyramid
        py_img2 = Gaussian_Pyramid(img2,no_of_levels,k,sig)
    elif pyramid == 'Laplacian':
        py_img1 = Laplacian_Pyramid(img1,no_of_levels,k,sig)                #Laplacian pyramid
        py_img2 = Laplacian_Pyramid(img2,no_of_levels,k,sig)    
    a,b = img1.shape
    U_prev = np.zeros((a,b,2))                                              #U_mat initialization
    for i in range(no_of_levels-1,-1,-1):                                   # i ---level number
        a,b = py_img1[i].shape                                              
        U_prev = cv2.resize(U_prev,(b,a))                                   # resizing U_prev to move it to next level
        U_prev ,img1_mod = planar_flow(py_img1[i],py_img2[i],U_prev,f,t,iter=iter_per_level) #planar flow per level
        print(i)                                                                #printing the level number                                              
    return U_prev,img1_mod                                                      #returning U_prev and modified image

RUN THE CODE FROM HERE

In [ ]:
img1 = cv2.imread('/content/drive/Shared drives/CV_Project/Dataset/planar_flow/2.jpg') #input images
img2 = cv2.imread('/content/drive/Shared drives/CV_Project/Dataset/planar_flow/2_mod.jpg')
img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)                                        #Converting them to grayscale
img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
img1,img2 = downsamp(img1,img2)                                                     #downsampling the image to reduce the compuational complexity
f = 3.6/1000                    #focal length 
t = np.array([[2],[1],[1]])         #translation vector
U_prev, img1_mod = hier_planar(img1,img2,f,t,no_of_levels=3,pyramid = 'Gaussian',iter_per_level=6) #hierarchical planar flow using gaussian pyramids

In [ ]:
#Calculating the modified image based on U_prev obtained from gaussian pyramids
out = img_modification(img2,img1,U_prev)
plt.figure(figsize=(8,11))
plt.imshow(out,cmap='gray')

In [ ]:
#Raw difference
diff = cv2.subtract(img1,img2)
plt.figure(figsize=(8,11))
plt.imshow(diff,cmap = 'gray')

In [ ]:
#compensated difference
diff = cv2.subtract(out,img2)
plt.figure(figsize=(8,11))
plt.imshow(diff,cmap = 'gray')

In [ ]:
# Hierarchical planar flow estimation using laplacian pyramids
t = np.array([[2],[1],[1]])
U_prev, img1_mod = hier_planar(img1,img2,f,t,no_of_levels=3,pyramid = 'Laplacian',iter_per_level=6)
out = img_modification(img2,img1,U_prev)
plt.figure(figsize=(8,11))
plt.imshow(out,cmap='gray')

In [ ]:
#compensated image difference
diff = cv2.subtract(out,img2)
plt.figure(figsize=(8,11))
plt.imshow(diff,cmap = 'gray')